In [180]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from sklearn.model_selection import train_test_split
from spektral.layers import GCNConv
from spektral.transforms import LayerPreprocess, AdjToSpTensor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from models import *
import numpy as np
from scipy.sparse import csr_matrix
from spektral.utils.sparse import sp_matrix_to_sp_tensor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [224]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

def load_task(dataset="pubmed", n_splits=3, n_labels=3, feature_dim=None):
  dataset = dataset.lower()

  if dataset in ["pubmed"]:
    dataset = Citation(dataset, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

  graph = dataset[0]
  x, adj, y = graph.x, graph.a, graph.y

  # split dataset for continual learning
  size = x.shape[0]
  idx_target = np.random.randint(low=0, high=n_splits, size=size)
  tasks = dict()

  for i in range(n_splits):
    idx = np.where(idx_target==i)[0]
    csr_adj = csr_matrix(adj[idx,:][:,idx])
    sparse_adj = sp_matrix_to_sp_tensor(csr_adj)

    y_sliced = y[idx,:].argmax(axis=1)
    cur_idx = np.arange(y_sliced.shape[0])

    idx_tr, idx_te, _, y_te = train_test_split(cur_idx, y_sliced, train_size=0.6, stratify=y_sliced)
    idx_va, idx_te = train_test_split(idx_te, train_size=0.2, stratify=y_te)
        
    tasks[i] = {"data": (x[idx,:], sparse_adj, y[idx,:]),
                "masks": (cur_idx[idx_tr], cur_idx[idx_va], cur_idx[idx_te])}

  return tasks

In [213]:
tasks = load_task()

Pre-processing node features


C:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


(6624,) (3974,) (530,) (2120,)
(6492,) (3895,) (519,) (2078,)
(6601,) (3960,) (528,) (2113,)


In [194]:
(x, a, y), (tr, va, te) = tasks[0].values()
x.shape, a.shape, y.shape, tr.shape, va.shape, te.shape

((6481, 500), TensorShape([6481, 6481]), (6481, 3), (6481,), (6481,), (6481,))

In [19]:
def apx_fisher(model, inputs):
  """ Approximate the fisher matrices of the model parameters """
  fisher_matrices = {p: tf.zeros_like(v.value()) for p,v in enumerate(model.trainable_variables)}

  with tf.GradientTape() as tape:
    pred = model(inputs)

    # get loglikelihood of predictions
    loglike = tf.nn.log_softmax(pred)
  
  # compute gradient wrt loglikelihood
  loglike_grads = tape.gradient(loglike, model.trainable_variables)

  # approximate i-th Fisher matrix with gradients squared
  for i,g in enumerate(loglike_grads):
    fisher_matrices[i] += tf.reduce_mean(g**2, axis=0)
  
  return fisher_matrices

  def penalty_loss(fisher_matrix, current_weights, prior_weights, lambda_=0.1):
    loss = 0

    for u, v, w in zip(fisher_matrix, current_weights, prior_weights):
        loss += tf.reduce_sum(u*tf.square(v-w))
    
    return 0.5 * lambda_ * loss

In [241]:
from os import listdir

class ModelEval():
    def __init__(self, dataset="pubmed", n_labels=2, n_splits=3, print_freq=10):
        super(ModelEval, self).__init__()

        self.n_labels = n_labels
        dataset = dataset.lower()
        self.print_freq = print_freq
        self.tasks = load_task(dataset=dataset, n_splits=n_splits)
        self.available_tasks = n_splits
    
    def train(self, task_num, epochs=20, continual_learning=False, use_ewc=False,
              lr=0.001, lambda_=0.1, verbose=True):
        task = self.tasks[task_num]
        x, adj, y = task["data"]
        mask_tr, _, _ = task["masks"]

        # initialize model if not already trained
        if not continual_learning:
            self.model = GNNNodeClassifier(n_labels=self.n_labels)
            self.optim = Adam(learning_rate=lr)
            self.loss_fn = CategoricalCrossentropy()
        
        self.accuracy = CategoricalAccuracy()
        
        for i in range(epochs):
            with tf.GradientTape() as tape:
                pred = self.model([x, adj])
                loss = self.loss_fn(y[mask_tr,:], pred[mask_tr,:])
            
            self.accuracy.update_state(y_pred=pred, y_true=y)

            # if continual learning and EWC is enabled add l2 loss wrt previous weights
            if continual_learning and use_ewc:
                loss += penalty_loss(self.fisher_matrix, self.model.get_weights(),
                                        self.prior_weights, lambda_=lambda_)
            
            gradients = tape.gradient(loss, self.model.trainable_variables)
            self.optim.apply_gradients(zip(gradients, self.model.trainable_variables))

            acc = self.accuracy.result().numpy()
            
            if i % self.print_freq == 0 and verbose:
                print(f"epoch={i}, train_loss={loss:.3f}, train_acc={acc:.3f}")

        self.prior_weights = self.model.get_weights()
        self.fisher_matrix = apx_fisher(self.model, [x, adj])
    
    def test(self, task_num, verbose=True):
        task = self.tasks[task_num]
        x, adj, y = task["data"]
        _, _, mask_te = task["masks"]


        pred = self.model([x, adj])
        acc = CategoricalAccuracy()(y_pred=pred[mask_te], y_true=y[mask_te]).numpy()

        if verbose:
            print(f"test_acc={acc:.3f}")

        return acc

In [242]:
model_eval = ModelEval()
n_tasks = 3
epochs = 50

for i in range(n_tasks):
    model_eval.train(i, epochs=epochs, continual_learning=False, verbose=False)
    acc = model_eval.test(i, verbose=False)
    print(f"task={i}, test_acc={acc}")

Pre-processing node features


C:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([4192,  321,  658, ..., 6161, 4041, 5129])

In [235]:
dataset = "citeseer",
model_eval = ModelEval()
model_eval.add_task(dataset)
model_eval.train(dataset=dataset, epochs=50)
acc = model_eval.test(dataset)

Pre-processing node features


C:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


AttributeError: 'ModelEval' object has no attribute 'add_task'

In [95]:
model_eval.add_task(dataset="citeseer")

Pre-processing node features


C:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\spektral\datasets\citation.py:194: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
C:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [362]:
model_eval.train("citeseer", epochs=50, continual_learning=True)
print("Accuracy after training on Pubmed and Cite")
print("Accuracy on Citeseer after training on Pubmed+Citeseer: ", model_eval.test("citeseer"))
print("Accuracy on Pubmed")

model_eval.train("RU", cl=True, with_ewc=False, lambda_=0.1, epochs=100)
print("Accuracy on RU after training on DE+FR w/o EWC: ", model_eval.test("RU"))
print("Accuracy on DE after training on DE+FR w/o EWC: ", model_eval.test("DE"))

Accuracy on DE after training on DE:  0.6042105263157894
Accuracy on RU after training on DE+FR w/o EWC:  0.7548460661345496
Accuracy on DE after training on DE+FR w/o EWC:  0.3957894736842105


In [364]:
model_eval = ModelEval()
model_eval.train("DE", epochs=50)
model_eval.train("RU", cl=True, with_ewc=True, lambda_=0.1, epochs=50)
print("Accuracy on FR after training on DE+FR w EWC: ", model_eval.test("DE"))
print("Accuracy on DE after training on DE+FR w EWC: ", model_eval.test("RU"))

1.0
Accuracy on FR after training on DE+FR w EWC:  0.6047368421052631
1.0
Accuracy on DE after training on DE+FR w EWC:  0.2451539338654504


In [251]:
model_eval = ModelEval()
model_eval.train("FR")
print("Accuracy on FR after training on FR w EWC: ", model_eval.test("RU"))

Accuracy on FR after training on FR w EWC:  0.7548460661345496


In [253]:
print("Accuracy on FR after training on DE+FR w EWC: ", model_eval.test("FR"))

Accuracy on FR after training on DE+FR w EWC:  0.6312977099236641


In [264]:
data, _ = load_task("DE", path="./data/twitch/")

In [269]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
pca.fit_transform(data[0]).shape

(9498, 100)

In [56]:
(x, adj, y), (mask_tr, _, mask_te) = load_task(language="DE", path="./data/twitch/", dataset="Twitch")

In [52]:
mask_tr.sum() / len(mask_tr), mask_te.sum() / len(mask_te)

(0.5999157717414193, 0.20004211412929038)

In [66]:
tf.reduce_sum(tf.sparse.to_dense(adj)[0,:])

<tf.Tensor: shape=(), dtype=float64, numpy=0.19624381498210133>

In [366]:
from spektral.datasets.graphsage import GraphSage
x, adj, y, mask_tr, mask_va, mask_te = GraphSage("ppi")

ConnectionError: HTTPConnectionPool(host='snap.stanford.edu', port=80): Max retries exceeded with url: /graphsage/ppi.zip (Caused by NewConnectionError("<urllib3.connection.HTTPConnection object at 0x000001F768E8C940>: Failed to establish a new connection: [WinError 10060] Impossibile stabilire la connessione. Risposta non corretta della parte connessa dopo l'intervallo di tempo oppure mancata risposta dall'host collegato"))

In [373]:
from dataloader import Git

data = Git(path="./data/git_web_ml/")

37700 37700


In [375]:
data[0]

Graph(n_nodes=37700, n_node_features=4004, n_edge_features=None, n_labels=2)